In [1]:
# Imports
import requests
from bs4 import BeautifulSoup
import psycopg2

In [2]:
# Variablen
conn_Ok = True


In [3]:
# Check if connection to DB possible
try:
    connection = psycopg2.connect(
        dbname='PSQL_ADSFS2023Gruppe15',
        user='ADSFS2023Gruppe15',
        password='ADS_FS_2023_G15!?',
        host='localhost',
        port='5432'
    )

    cursor = connection.cursor()

except (Exception, psycopg2.Error) as error:
        print('Fehler beim Verbinden mit der PostgreSQL-Datenbank:', error)
        conn_Ok = False

In [4]:
def get_team_id(team_name):

    if team_name == 'Bor. Mönchengladbach':
        team_name = 'Borussia Mönchengladbach'
    elif team_name == '1899 Hoffenheim':
        team_name = 'TSG 1899 Hoffenheim'
    elif team_name == 'SV Werder Bremen':
        team_name = 'Werder Bremen'
   
        
    # Vergleicht Team Namen und sucht nach ID
    cursor.execute("SELECT team_id FROM bundesliga_mannschaften WHERE mannschaft = %s", (team_name,))
    result = cursor.fetchone()[0]

    if result:
        return result
    else:
        print(f'Keine Übereinstimmung für {team_name} gefunden')


In [5]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-ballkontakte/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        touch_elements = soup.find_all('td', class_='team_stats-balls_touched team_stats-balls_touched-list')

        for team_element, touch_element in zip(team_elements, touch_elements):
            team_name = team_element.text.strip()
            touches_total = touch_element.text.strip()

            insert_data_query = '''
                INSERT INTO bundesliga_touch_stats (matchday, team_id, touches_total)
                VALUES (%s, %s, %s);
            '''
            team_id = get_team_id(team_name)

            cursor.execute(insert_data_query, (matchday, team_id, touches_total))

            connection.commit()

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Torschüsse')

65 1. FC Köln
40 FC Bayern München
87 Borussia Mönchengladbach
1635 RB Leipzig
131 VfL Wolfsburg
129 VfL Bochum
6 Bayer Leverkusen
7 Borussia Dortmund
54 Hertha BSC
95 FC Augsburg
80 1. FC Union Berlin
134 Werder Bremen
81 1. FSV Mainz 05
91 Eintracht Frankfurt
175 TSG 1899 Hoffenheim
112 SC Freiburg
16 VfB Stuttgart
9 FC Schalke 04
40 FC Bayern München
87 Borussia Mönchengladbach
134 Werder Bremen
6 Bayer Leverkusen
7 Borussia Dortmund
91 Eintracht Frankfurt
175 TSG 1899 Hoffenheim
1635 RB Leipzig
16 VfB Stuttgart
81 1. FSV Mainz 05
80 1. FC Union Berlin
65 1. FC Köln
129 VfL Bochum
54 Hertha BSC
112 SC Freiburg
131 VfL Wolfsburg
95 FC Augsburg
9 FC Schalke 04
1635 RB Leipzig
131 VfL Wolfsburg
87 Borussia Mönchengladbach
16 VfB Stuttgart
40 FC Bayern München
134 Werder Bremen
91 Eintracht Frankfurt
6 Bayer Leverkusen
7 Borussia Dortmund
65 1. FC Köln
81 1. FSV Mainz 05
95 FC Augsburg
112 SC Freiburg
175 TSG 1899 Hoffenheim
54 Hertha BSC
129 VfL Bochum
9 FC Schalke 04
80 1. FC Union Be

In [6]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-laufleistung/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        distance_elements = soup.find_all('td', class_='team_stats-tracking_distance team_stats-tracking_distance-format')
        try:
            for team_element, distance_element in zip(team_elements, distance_elements):
                team_name = team_element.text.strip()
                distance_total = distance_element.text.strip()

                insert_data_query = '''
                    INSERT INTO bundesliga_distance_stats (matchday, team_id, distance_total)
                    VALUES (%s, %s, %s);
                    '''
                team_id = get_team_id(team_name)
            
                cursor.execute(insert_data_query, (matchday, team_id, float(distance_total.replace(',', '.'))))
                            
                connection.commit()
        except Exception as e:
            connection.rollback
            print(f'Fehler beim Einfügen der Daten für Spieltag {matchday} und Laufleistung:', str(e))

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Laufleistung')

117,55
65 1. FC Köln
117,16
81 1. FSV Mainz 05
116,41
40 FC Bayern München
114,55
7 Borussia Dortmund
114,05
16 VfB Stuttgart
113,89
91 Eintracht Frankfurt
113,20
129 VfL Bochum
112,81
80 1. FC Union Berlin
112,05
1635 RB Leipzig
111,88
6 Bayer Leverkusen
111,88
134 Werder Bremen
111,73
112 SC Freiburg
110,86
9 FC Schalke 04
110,21
131 VfL Wolfsburg
108,64
87 Borussia Mönchengladbach
108,36
95 FC Augsburg
107,75
54 Hertha BSC
104,96
175 TSG 1899 Hoffenheim
118,13
95 FC Augsburg
117,77
112 SC Freiburg
116,56
7 Borussia Dortmund
115,92
65 1. FC Köln
115,68
175 TSG 1899 Hoffenheim
115,41
134 Werder Bremen
114,82
9 FC Schalke 04
114,01
40 FC Bayern München
113,70
16 VfB Stuttgart
112,90
80 1. FC Union Berlin
112,78
131 VfL Wolfsburg
112,22
54 Hertha BSC
111,24
6 Bayer Leverkusen
111,04
91 Eintracht Frankfurt
110,25
129 VfL Bochum
107,65
81 1. FSV Mainz 05
106,53
87 Borussia Mönchengladbach
105,54
1635 RB Leipzig
121,11
80 1. FC Union Berlin
119,87
112 SC Freiburg
119,21
65 1. FC Köln
117,7

In [7]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-freistoesse/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        freekicks_elements = soup.find_all('td', class_='team_stats-freekicks team_stats-freekicks-list')

        for team_element, freekicks_element in zip(team_elements, freekicks_elements):
            team_name = team_element.text.strip()
            freekicks_total = freekicks_element.text.strip()

            insert_data_query = '''
                INSERT INTO bundesliga_freekicks (matchday, team_id, freekicks_total)
                VALUES (%s, %s, %s);
                '''
            team_id = get_team_id(team_name)
        
            cursor.execute(insert_data_query, (matchday, team_id, freekicks_total))
                        
            connection.commit()

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Laufleistung')

54 Hertha BSC
175 TSG 1899 Hoffenheim
80 1. FC Union Berlin
131 VfL Wolfsburg
6 Bayer Leverkusen
7 Borussia Dortmund
95 FC Augsburg
129 VfL Bochum
1635 RB Leipzig
40 FC Bayern München
87 Borussia Mönchengladbach
91 Eintracht Frankfurt
112 SC Freiburg
81 1. FSV Mainz 05
134 Werder Bremen
65 1. FC Köln
16 VfB Stuttgart
9 FC Schalke 04
7 Borussia Dortmund
1635 RB Leipzig
6 Bayer Leverkusen
81 1. FSV Mainz 05
87 Borussia Mönchengladbach
54 Hertha BSC
131 VfL Wolfsburg
40 FC Bayern München
91 Eintracht Frankfurt
112 SC Freiburg
134 Werder Bremen
80 1. FC Union Berlin
95 FC Augsburg
9 FC Schalke 04
65 1. FC Köln
16 VfB Stuttgart
95 FC Augsburg
6 Bayer Leverkusen
134 Werder Bremen
65 1. FC Köln
1635 RB Leipzig
131 VfL Wolfsburg
175 TSG 1899 Hoffenheim
91 Eintracht Frankfurt
16 VfB Stuttgart
87 Borussia Mönchengladbach
9 FC Schalke 04
81 1. FSV Mainz 05
40 FC Bayern München
7 Borussia Dortmund
80 1. FC Union Berlin
129 VfL Bochum
54 Hertha BSC
112 SC Freiburg
134 Werder Bremen
87 Borussia Mönc

In [8]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-eckbaelle/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        corner_left_elements = soup.find_all('td', class_='team_stats-corner_kicks_left team_stats-corner_kicks_left-list')
        corner_right_elements = soup.find_all('td', class_='team_stats-corner_kicks_right team_stats-corner_kicks_right-list')
        corner_total_elements = soup.find_all('td', class_='team_stats-corner_kicks team_stats-corner_kicks-list hs-sorted-value')

        for team_element, corner_left_element, corner_right_element, corner_total_element in zip(team_elements, corner_left_elements, corner_right_elements, corner_total_elements):
            team_name = team_element.text.strip()
            corner_left = corner_left_element.text.strip()
            corner_right = corner_right_element.text.strip()
            corner_total = corner_total_element.text.strip()

            insert_data_query = '''
                INSERT INTO bundesliga_corners (matchday, team_id, corner_left, corner_right, corner_total)
                VALUES (%s, %s, %s, %s, %s);
                '''
            team_id =get_team_id(team_name)
        
            cursor.execute(insert_data_query, (matchday, team_id, corner_left, corner_right, corner_total))
                        
            connection.commit()

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Laufleistung')